### Importing all the necessary libraries

In [2]:
import os
import glob
import cv2
import json
import pathlib
import numpy as np
import pandas as pd
import earthpy.plot as ep
import earthpy.spatial as es
from matplotlib import pyplot as plt
%matplotlib inline
from progressbar import ProgressBar
from dataset import readRas
from mrc_insar_common.data import data_reader
from config import get_config

### Get the hyperparameter dictionary from `config` file using `get_config()` function and creating directory for saving the figures.

In [3]:
config = get_config()
pathlib.Path(config['visualization_dir']).mkdir(parents=True, exist_ok=True)

### Helper Functions

#### For ploting different type of images.

plot():

- display_list (dict) - a dictionary where keys are the title of the image and values are images.
- id (int) - image id.
- directory (str) - path to save the plot.

In [4]:
def plot(display_list, id, directory, show=True):
    plt.figure(figsize=(20, 16))
    title = list(display_list.keys())
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)

        # plot dem channel using earthpy
        if title[i]=="dem":
            ax = plt.gca()
            hillshade = es.hillshade(display_list[title[i]], azimuth=180)
            ep.plot_bands(
                display_list[title[i]],
                cbar=False,
                cmap="terrain",
                title=title[i],
                ax=ax
            )
            ax.imshow(hillshade, cmap="Greys", alpha=0.5)
        # gray image plot rslc\n",
        elif title[i]=="rslc1" or title[i]=="rslc2":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]])**0.25, cmap="gray", interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        elif title[i]=="rslc1_label" or title[i]=="rslc2_label":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]])**0.3, cmap="gray", interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        # ifr plot
        elif title[i]=="ifg":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]]), cmap='YlGnBu', interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        # plot labels
        else:
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]]), vmin=0, vmax=1)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
    
    prediction_name = "img_id_{}.png".format(id) # create file name to save
    plt.savefig(os.path.join(directory, prediction_name), bbox_inches='tight', dpi=800)
    if show:
        plt.show()
    plt.clf()
    plt.cla()
    plt.close()

#### For calculating total pixel ratio

In [5]:
def total_pixel(config):
    label = sorted(glob.glob(config["dataset_dir"]+"label/*.ras"))
    rslc1_label = readRas(label[0])[0]

    rslc1_label[rslc1_label==255] = 1
    total_pix = rslc1_label.shape[0]*rslc1_label.shape[1]
    pixels = {"Water":np.sum(rslc1_label)/total_pix, 
              "NON-Water":((rslc1_label.shape[0]*rslc1_label.shape[1]) - np.sum(rslc1_label))/total_pix}
    return pixels
    
total_pixel(config)

{'Water': 0.23953188267219586, 'NON-Water': 0.7604681173278042}

### Plotting Row Data

In [6]:
def display_raw_data(config):
    
    pathlib.Path((config['visualization_dir']+'display_raw_data')).mkdir(parents = True, exist_ok = True)

    wid, hgt = 11361, 10820  # from the masterpar.xml

    rslcs = sorted(glob.glob(config["dataset_dir"]+"rslc/*.rslc.notopo"))
    label = sorted(glob.glob(config["dataset_dir"]+"label/*.ras"))

    rslc1 = data_reader.readBin(rslcs[0], width=wid, dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(rslcs[1], width=wid, dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(label[0])[0]
    rslc2_label = readRas(label[1])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1

    id = rslcs[0].split("/")[-1].split(".")[0].split("\\")[-1]
    id2 = rslcs[3].split("/")[-1].split(".")[0].split("\\")[-1]

    display_list = {
                     "rslc1": np.abs(rslc1),
                     "rslc2": np.abs(rslc2),
                     "ifg": np.angle(rslc2*np.conjugate(rslc1)),
                     "rslc1_label": rslc1_label,
                     "rslc2_label": rslc2_label
                     }
    plot(display_list, (id+"_"+id2), (config['visualization_dir']+'display_raw_data'))

display_raw_data(config)

### Plotting Tiles

In [7]:
def display_all(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """
    
    pathlib.Path((config['visualization_dir']+'display_tiles')).mkdir(parents = True, exist_ok = True)

    
    rslc1 = data_reader.readBin(data['rslc0'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    ifg = np.angle(np.log(np.abs(rslc2))*np.conjugate(np.log(np.abs(rslc1))))

    display_list = {
                   "rslc1":np.abs(rslc1),
                   "rslc2":np.abs(rslc2),
                   "ifg":ifg,
                   "rslc1_label": rslc1_label,
                   "rslc2_label":rslc2_label
                  }
    plot(display_list, id, (config['visualization_dir']+'display_tiles'))


# change the value for specific plot
train_index = 1
val_index = 1
test_index = 1

train_dir = pd.read_csv(config['train_dir'])
print("Train examples: ", len(train_dir))
display_all(train_dir.iloc[train_index], config)
    
valid_dir = pd.read_csv(config['valid_dir'])
print("Valid examples: ", len(valid_dir))
display_all(valid_dir.iloc[val_index], config)

test_dir = pd.read_csv(config['test_dir'])
print("Test examples: ", len(test_dir))
display_all(test_dir.iloc[test_index], config)

Train examples:  404
Valid examples:  50
Test examples:  51


### Plotting Patch image

#### Ploting patch image of size (512 X 512)

In [ ]:
def display_patch(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """

    pathlib.Path((config['visualization_dir']+'display_patch_512')).mkdir(parents = True, exist_ok = True)

    rslc1 = data_reader.readBin(data['rslc0'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    rslc1_amp = rslc1[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_amp = rslc2[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc1_label_amp = rslc1_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_label_amp = rslc2_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
        
    display_list = {
                    "rslc1":np.abs(rslc1_amp),
                    "rslc2":np.abs(rslc2_amp),
                    "ifg":ifg,
                    "rslc1_label": rslc1_label_amp,
                    "rslc2_label":rslc2_label_amp
                    }
    plot(display_list, id, (config['visualization_dir']+'display_patch_512'))
            
# change the value for specific plot
train_index = 1
val_index = 1
test_index = 1


with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df.iloc[train_index], config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df.iloc[val_index], config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
test_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Test examples: ",len(test_patch_df))
display_patch(test_patch_df.iloc[test_index], config)

##### Run the below cell to save all the patch images

In [ ]:
def display_patch(data, config):
        """
        Summary:
            save all images into single figure
        Arguments:
            data : data file holding images path
            directory (str) : path to save images
        Return:
            save images figure into directory
        """
        
        pathlib.Path((config['visualization_dir']+'display_patch_512')).mkdir(parents = True, exist_ok = True)
        pbar = ProgressBar()
        for i in pbar(range(len(data))):
            rslc1 = data_reader.readBin(data['rslc0'][i], width=512, dataType='floatComplex') # SLC(RSLC) image
            rslc2 = data_reader.readBin(data['rslc1'][i], width=512, dataType='floatComplex') # SLC(RSLC) image
            rslc1_label = readRas(data['rslc0_label'][i])[0]
            rslc2_label = readRas(data['rslc1_label'][i])[0]
    
            rslc1_label[rslc1_label==255] = 1
            rslc2_label[rslc2_label==255] = 1
            
            rslc1_amp = rslc1[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_amp = rslc2[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc1_label_amp = rslc1_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_label_amp = rslc2_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            
            id = data['rslc1'][i].split("/")[-1].split(".")[0].split("\\")[-1]
            ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
            
            display_list = {
                            "rslc1":np.abs(rslc1_amp),
                            "rslc2":np.abs(rslc2_amp),
                            "ifg":ifg,
                            "rslc1_label": rslc1_label_amp,
                            "rslc2_label":rslc2_label_amp
                            }
            plot(display_list, id, (config['visualization_dir']+'display_patch_512'), show=False)
            
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
test_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Test examples: ",len(test_patch_df))
display_patch(test_patch_df, config)

#### Ploting patch balance image of size (512 X 512)

In [8]:
def display_patch(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """

    pathlib.Path((config['visualization_dir']+'display_patch_512')).mkdir(parents = True, exist_ok = True)

    rslc1 = data_reader.readBin(data['rslc0'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=512, dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    rslc1_amp = rslc1[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_amp = rslc2[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc1_label_amp = rslc1_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_label_amp = rslc2_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
        
    display_list = {
                    "rslc1":np.abs(rslc1_amp),
                    "rslc2":np.abs(rslc2_amp),
                    "ifg":ifg,
                    "rslc1_label": rslc1_label_amp,
                    "rslc2_label":rslc2_label_amp
                    }
    plot(display_list, id, (config['visualization_dir']+'display_patch_512'))
            
# change the value for specific plot
train_index = 1
val_index = 1
test_index = 1


with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/train_patch_phr_cb_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df.iloc[train_index], config)
    
with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/valid_patch_phr_cb_256.json", 'r') as j:
        patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df.iloc[val_index], config)
    
# with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
#         patch_dir = json.loads(j.read())
# test_patch_df = pd.DataFrame.from_dict(patch_dir)
# print("Test examples: ",len(test_patch_df))
# display_patch(test_patch_df.iloc[test_index], config)

Train examples:  134
Valid examples:  191


##### Run the below cell to save all the patch balance images

In [ ]:
def display_patch(data, config):
        """
        Summary:
            save all images into single figure
        Arguments:
            data : data file holding images path
            directory (str) : path to save images
        Return:
            save images figure into directory
        """
        
        pathlib.Path((config['visualization_dir']+'display_patch_512')).mkdir(parents = True, exist_ok = True)
        pbar = ProgressBar()
        for i in pbar(range(len(data))):
            rslc1 = data_reader.readBin(data['rslc0'][i], width=512, dataType='floatComplex') # SLC(RSLC) image
            rslc2 = data_reader.readBin(data['rslc1'][i], width=512, dataType='floatComplex') # SLC(RSLC) image
            rslc1_label = readRas(data['rslc0_label'][i])[0]
            rslc2_label = readRas(data['rslc1_label'][i])[0]
    
            rslc1_label[rslc1_label==255] = 1
            rslc2_label[rslc2_label==255] = 1
            
            rslc1_amp = rslc1[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_amp = rslc2[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc1_label_amp = rslc1_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_label_amp = rslc2_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            
            id = data['rslc1'][i].split("/")[-1].split(".")[0].split("\\")[-1]
            ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
            
            display_list = {
                            "rslc1":np.abs(rslc1_amp),
                            "rslc2":np.abs(rslc2_amp),
                            "ifg":ifg,
                            "rslc1_label": rslc1_label_amp,
                            "rslc2_label":rslc2_label_amp
                            }
            plot(display_list, id, (config['visualization_dir']+'display_patch_512'), show=False)
            
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
test_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Test examples: ",len(test_patch_df))
display_patch(test_patch_df, config)